In [14]:
import timm
import torch
import torch.nn.functional as F

import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [15]:
from configs.config_v1 import CFG

In [18]:
TEST1_FOLDER = 'data/public/videos/'
TEST2_FOLDER = 'data/public_test_2/videos/'

In [16]:
model = timm.create_model('tf_efficientnet_b0', pretrained=True, features_only=True)
model.eval()
print()

In [17]:
def get_emb(model, inp):
    with torch.no_grad():
        out = model(inp)
        emb = F.adaptive_avg_pool2d(out[-1], 1)[0,:,0,0].numpy()
    return emb

def get_frame(vid_name, vid_dir=CFG.train_video_dir, frame_ind=0):
    vid_path = os.path.join(vid_dir, vid_name)
    cap = cv2.VideoCapture(vid_path)
    cap.set(1, frame_ind)  # Where frame_no is the frame you want
    ret, im = cap.read()
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    return im

def get_tensor(im):
    inp = CFG.val_transforms(image=im)['image']
    inp = inp.unsqueeze(0)
    return inp

def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [10]:
train_df = pd.read_csv('data/train/label.csv')

frame_counts = []

all_embeddings = []
for fname in tqdm(train_df.fname.tolist()):
    
    vid_name1 = fname
    im1 = get_frame(vid_name1)
    inp1 = get_tensor(im1)
    emb1 = get_emb(model, inp1)
    all_embeddings.append(emb1)
    # break

  0%|          | 0/1168 [00:00<?, ?it/s]

In [11]:
cat_embeddings = np.stack(all_embeddings)

In [12]:
np.save('data/train_embeddings_frame0', cat_embeddings)

In [27]:
public_test_df = pd.DataFrame(os.listdir(TEST1_FOLDER))
public_test_df.columns = ['fname']

frame_counts = []

all_embeddings = []
for fname in tqdm(public_test_df.fname.tolist()):
    vid_name1 = fname
    im1 = get_frame(vid_name1, vid_dir=TEST1_FOLDER)
    inp1 = get_tensor(im1)
    emb1 = get_emb(model, inp1)
    all_embeddings.append(emb1)
    # break

  0%|          | 0/350 [00:00<?, ?it/s]

In [28]:
cat_embeddings = np.stack(all_embeddings)

In [29]:
np.save('data/public_embeddings_frame0', cat_embeddings)

In [30]:
public_test_2_df = pd.DataFrame(os.listdir(TEST2_FOLDER))
public_test_2_df.columns = ['fname']

frame_counts = []

all_embeddings = []
for fname in tqdm(public_test_2_df.fname.tolist()):
    vid_name1 = fname
    im1 = get_frame(vid_name1, vid_dir=TEST2_FOLDER)
    inp1 = get_tensor(im1)
    emb1 = get_emb(model, inp1)
    all_embeddings.append(emb1)
    # break

  0%|          | 0/486 [00:00<?, ?it/s]

In [31]:
cat_embeddings = np.stack(all_embeddings)

In [32]:
np.save('data/public_test_2_embeddings_frame0', cat_embeddings)